In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [198]:
import pandas as pd
path='/content/drive/MyDrive/Colab Notebooks/5002/data/Data_Q5/'

# Load the provided files
train_file_path =path+'Q5_train.xlsx'
test_file_path =path+'/Q5_test.csv'
sample_submission_path =path+'Q5_sample_submission.csv'

# Reading the files
train_data = pd.read_excel(train_file_path)
test_data = pd.read_csv(test_file_path)
sample_submission = pd.read_csv(sample_submission_path)

In [199]:
train_data.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165.0,60,81.0,0.5,0.6,1,1.0,135,...,40,75,16.5,1.0,1.0,22,25,27,0,1
1,1,70,165.0,65,89.0,0.6,0.7,2,2.0,146,...,57,126,16.2,1.0,1.1,27,23,37,1,0
2,2,20,170.0,75,81.0,0.4,0.5,1,1.0,118,...,45,93,17.4,1.0,0.8,27,31,53,0,1
3,3,35,180.0,95,105.0,1.5,1.2,1,1.0,131,...,38,102,15.9,1.0,1.0,20,27,30,1,0
4,4,30,165.0,60,80.5,1.5,1.0,1,1.0,121,...,44,93,15.4,1.0,0.8,19,13,17,0,1


In [200]:
len(train_data),len(test_data)

(159256, 106171)

In [213]:
#train_data.info()

In [202]:
missing_values=train_data.isnull().sum()
missing_values

id                        0
age                       0
height(cm)             9555
weight(kg)                0
waist(cm)                 8
eyesight(left)         9564
eyesight(right)           9
hearing(left)             0
hearing(right)         9555
systolic                  0
relaxation                0
fasting blood sugar       0
Cholesterol               0
triglyceride              0
HDL                       0
LDL                       0
hemoglobin                0
Urine protein          9587
serum creatinine          0
AST                       0
ALT                       0
Gtp                       0
dental caries             0
smoking                   0
dtype: int64

In [203]:
#train_data.describe().T

In [204]:
# Check Duplicated Values
train_data.duplicated().sum()

0

In [205]:
unique_values_per_column = train_data.nunique()
sorted_unique_values = unique_values_per_column.sort_values(ascending=False)
print(f"{sorted_unique_values} unique values")

id                     159256
waist(cm)                 531
triglyceride              392
Gtp                       362
fasting blood sugar       229
Cholesterol               227
LDL                       222
ALT                       188
AST                       140
hemoglobin                134
systolic                  112
HDL                       108
relaxation                 75
serum creatinine           28
weight(kg)                 28
eyesight(left)             20
age                        18
eyesight(right)            17
height(cm)                 14
Urine protein               6
hearing(left)               2
hearing(right)              2
dental caries               2
smoking                     2
dtype: int64 unique values


## Data preprocessing

In [206]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
numerical_columns = train_data.select_dtypes(include=['int64', 'float64']).columns
train_data[numerical_columns] = imputer.fit_transform(train_data[numerical_columns])

In [207]:
import warnings
warnings.filterwarnings("ignore")

In [208]:
train_data = train_data[(train_data['eyesight(left)'] <= 2.0) & (train_data['eyesight(right)'] <= 2.0)]

In [215]:
train_data['BMI'] = train_data['weight(kg)'] / ((train_data['height(cm)'] / 100) ** 2)
train_data["Triglyceride_Rank"] = pd.cut(train_data["triglyceride"],bins = [0,150,199,499,train_data["triglyceride"].max()],labels = [1,2,3,4]).astype(int)
train_data["Cho_Rank"] = pd.cut(train_data["Cholesterol"],bins = [0,200,239,train_data["Cholesterol"].max()],
                        labels = [1,2,3]).astype(int)
train_data['Cholesterol_Lipit'] = train_data['Cholesterol'] / train_data['HDL']

train_data['highrisk'] = (train_data['age'] >= 45).astype(int)
train_data['lowhdl'] = (train_data['HDL'] <= 40).astype(int)
#train = train_data.drop(["eyesight(left)","eyesight(right)","hearing(left)","hearing(right)"],axis=1,inplace= False)
train=train_data

In [217]:
test_data['BMI'] = test_data['weight(kg)'] / ((test_data['height(cm)'] / 100) ** 2)
test_data["Triglyceride_Rank"] = pd.cut(test_data["triglyceride"],bins = [0,150,199,499,test_data["triglyceride"].max()],labels = [1,2,3,4]).astype(int)
test_data["Cho_Rank"] = pd.cut(test_data["Cholesterol"],bins = [0,200,239,test_data["Cholesterol"].max()],
                        labels = [1,2,3]).astype(int)
test_data['Cholesterol_Lipit'] = test_data['Cholesterol'] / test_data['HDL']

test_data['highrisk'] = (test_data['age'] >= 45).astype(int)
test_data['lowhdl'] = (test_data['HDL'] <= 40).astype(int)
#test = test_data.drop(["eyesight(left)","eyesight(right)","hearing(left)","hearing(right)"],axis=1,inplace= False)
test=test_data

In [216]:
train.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,ALT,Gtp,dental caries,smoking,BMI,Triglyceride_Rank,Cho_Rank,Cholesterol_Lipit,highrisk,lowhdl
0,0.0,55.0,165.0,60.0,81.0,0.5,0.6,1.0,1.0,135.0,...,25.0,27.0,0.0,1.0,22.038567,3,1,4.300000,1,1
1,1.0,70.0,165.0,65.0,89.0,0.6,0.7,2.0,2.0,146.0,...,23.0,37.0,1.0,0.0,23.875115,1,1,3.403509,1,0
2,2.0,20.0,170.0,75.0,81.0,0.4,0.5,1.0,1.0,118.0,...,31.0,53.0,0.0,1.0,25.951557,2,1,3.955556,0,0
3,3.0,35.0,180.0,95.0,105.0,1.5,1.2,1.0,1.0,131.0,...,27.0,30.0,1.0,0.0,29.320988,3,1,4.736842,0,1
4,4.0,30.0,165.0,60.0,80.5,1.5,1.0,1.0,1.0,121.0,...,13.0,17.0,0.0,1.0,22.038567,1,1,3.522727,0,0


In [209]:
num_cols = [col for col in train.columns if (train[col].dtype in ["int64","float64"]) & (train[col].nunique()>10)]

In [218]:
def corr_skew_outliner(df, cols):
    #outliner and #skewness part
    for col in cols:
        Q1 = df[col].quantile(0.01)
        Q3 = df[col].quantile(0.95)
        df.loc[df[col] < Q1, col] = Q1
        df.loc[df[col] > Q3, col] = Q3
    return df
train = corr_skew_outliner(train,num_cols)

## Model

In [219]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
X = train.drop(["smoking",'id'], axis=1)
y = train["smoking"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.15, random_state = 42)

In [238]:
class_counts = y_train.value_counts()
print(class_counts)

0.0    76003
1.0    59166
Name: smoking, dtype: int64


In [228]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
xgb_model = xgb.XGBClassifier(n_estimators=600, colsample_bytree=0.7, max_depth=5, learning_rate=0.1, random_state=42, scale_pos_weight=1.4)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict_proba(X_test)[:, 1]
roc_score_xgb = roc_auc_score(y_test, y_pred_xgb)
print(f"ROC AUC Score with XGBoost: {roc_score_xgb}")

ROC AUC Score with XGBoost: 0.87070475126009


In [237]:
import lightgbm as lgb
lgbm_model = lgb.LGBMClassifier(n_estimators=600, max_depth=5, learning_rate=0.1, random_state=42, scale_pos_weight=1.4)
lgbm_model.fit(X_train, y_train)
y_pred_proba_lgbm = lgbm_model.predict_proba(X_test)[:, 1]
roc_score_lgbm = roc_auc_score(y_test, y_pred_proba_lgbm)
print(f"ROC AUC Score with LightGBM: {roc_score_lgbm}")

ROC AUC Score with LightGBM: 0.8696467604174507


In [236]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(estimators=[('lgbm', lgbm_model),  ('xgb', xgb_model)], voting='soft')
voting.fit(X_train,y_train)
voting_pred = voting.predict_proba(X_test)[:,1]
print('Accuracy: ', roc_auc_score(y_test, voting_pred))

Accuracy:  0.8710017208676449


In [235]:
test.drop("id",axis=1, inplace=True)

In [233]:
sample_submission["smoking"]=voting.predict_proba(test)[:, 1]
sample_submission

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


,id,smoking
0,159256,0.699161
1,159257,0.339876
2,159258,0.472245
3,159259,0.027471
4,159260,0.609619
...,...,...
106166,265422,0.676401
106167,265423,0.725054
106168,265424,0.518584
106169,265425,0.141441


In [234]:
sample_submission.to_csv('Q5_output.csv',index=False)